In [1]:
import os
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from keras.models import Model, Sequential

def network(input_window_size=60, filter_number=32, conv_window=(3,), pooling_window=(2,), dropout_rate=[],
            activation='relu', dense_activation='softmax', optimizer='adam', loss='categorical_crossentropy', layers=1):
    model = Sequential()
    
    # Input Layer
    model.add(Conv1D(filter_number, conv_window, activation=activation, padding='same', input_shape=(input_window_size, 1)))
    model.add(MaxPooling1D(pooling_window, padding='same'))
    model.add(Dropout(dropout_rate[0]))

    
    # Hidden Layers
    current_filter = 1
    filter_number_temp = filter_number
    for i in range(layers):
        filter_number_temp = filter_number_temp *  2
        model.add(Conv1D(filter_number_temp, conv_window, activation=activation, padding='same'))
        model.add(MaxPooling1D(pooling_window, padding='same'))
        model.add(Dropout(dropout_rate[current_filter]))
        current_filter = current_filter + 1
    
    # Output Layer
    model.add(Flatten())
    model.add(Dense(2, activation=dense_activation))

    model.compile(optimizer=optimizer[0], loss=loss, metrics=['accuracy', 'categorical_accuracy'])

    return model

Using TensorFlow backend.


In [2]:
import os
import h5py
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def rolling_window(a, window, step_size, padding=True, copy=True):
    if copy:
        result = a.copy()
    else:
        result = a
    if padding:
        result = np.hstack((result, np.zeros(window)))
    shape = result.shape[:-1] + (result.shape[-1] - window + 1 - step_size, window)
    strides = result.strides + (result.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(result, shape=shape, strides=strides)


size = 60

base_path = '/home/irene/Documents/Datos-Tesis/Training_window_set_1'

total_files = len(os.listdir(base_path))
x_dataset = np.zeros(shape=(0, size))
y_dataset = np.zeros(shape=(0, 1))
count = 1
skipped = []
for file_path in os.listdir(base_path):
    current_index = 0
    
    h5f = h5py.File(os.path.join(base_path, file_path), 'r')
    
    x = rolling_window(h5f['normal'][:], size, 1)
    y = rolling_window(h5f['transformed'][:], size, 1)
    y = [[np.amax(array)] for array in y]

    x_dataset = np.vstack((x_dataset, x))
    y_dataset = np.vstack((y_dataset, y))
    
    print(count)
    count = count + 1

x_dataset = np.reshape(x_dataset, (len(x_dataset), size, 1))
y_dataset = np.reshape(y_dataset, (len(y_dataset), 1))
y_dataset = to_categorical(y_dataset)

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint, uniform
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

# Convert to scikit-learn model (build_fn is a function that returns a model)
model = KerasClassifier(build_fn=network)

# list of parameters
input_window_size = [60]
layers = [1]
filter_number = [32, 64]
conv_window = [(5,)]
pooling_window = [(2,)]
activation = ['relu']
dense_activation = ['softmax', 'sigmoid']
optimizer = [('adam', 0)]
loss = ['binary_crossentropy']
dropout_rate = [[0.1, 0.2, 0.1, 0.1]]

epochs= [30]
batch_size = [32]

param_dist = dict(input_window_size=input_window_size,
                  filter_number=filter_number, 
                  conv_window=conv_window, 
                  pooling_window=pooling_window,
                  dropout_rate = dropout_rate,
                  activation=activation, 
                  dense_activation=dense_activation, 
                  optimizer=optimizer, 
                  loss=loss, 
                  layers=layers,
                  epochs=epochs,
                  batch_size=batch_size)

# Random Search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=3, verbose=2, n_iter=4, n_jobs=-1)
result = random_search.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 196.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 196.9min finished


Instructions for updating:
Use tf.cast instead.
Epoch 1/30
221471/221471 [==============================] - 86s 388us/step - loss: 0.1552 - acc: 0.9551 - categorical_accuracy: 0.9551
Epoch 2/30
221471/221471 [==============================] - 86s 387us/step - loss: 0.1276 - acc: 0.9640 - categorical_accuracy: 0.9640
Epoch 3/30
221471/221471 [==============================] - 85s 385us/step - loss: 0.1181 - acc: 0.9668 - categorical_accuracy: 0.9668
Epoch 4/30
221471/221471 [==============================] - 89s 402us/step - loss: 0.1133 - acc: 0.9676 - categorical_accuracy: 0.9676
Epoch 5/30
221471/221471 [==============================] - 89s 402us/step - loss: 0.1106 - acc: 0.9683 - categorical_accuracy: 0.9683
Epoch 6/30
221471/221471 [==============================] - 89s 402us/step - loss: 0.1086 - acc: 0.9690 - categorical_accuracy: 0.9690
Epoch 7/30
221471/221471 [==============================] - 89s 403us/step - loss: 0.1063 - acc: 0.9696 - categorical_accuracy: 0.9696
Epoch 8

In [6]:
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means = result.cv_results_['mean_test_score']
stds = result.cv_results_['std_test_score']
params = result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.974385 using {'pooling_window': (2,), 'optimizer': ('adam', 0), 'loss': 'binary_crossentropy', 'layers': 1, 'input_window_size': 60, 'filter_number': 64, 'epochs': 30, 'dropout_rate': [0.1, 0.2, 0.1, 0.1], 'dense_activation': 'softmax', 'conv_window': (5,), 'batch_size': 32, 'activation': 'relu'}
0.972746 (0.001016) with: {'pooling_window': (2,), 'optimizer': ('adam', 0), 'loss': 'binary_crossentropy', 'layers': 1, 'input_window_size': 60, 'filter_number': 32, 'epochs': 30, 'dropout_rate': [0.1, 0.2, 0.1, 0.1], 'dense_activation': 'softmax', 'conv_window': (5,), 'batch_size': 32, 'activation': 'relu'}
0.974385 (0.000980) with: {'pooling_window': (2,), 'optimizer': ('adam', 0), 'loss': 'binary_crossentropy', 'layers': 1, 'input_window_size': 60, 'filter_number': 64, 'epochs': 30, 'dropout_rate': [0.1, 0.2, 0.1, 0.1], 'dense_activation': 'softmax', 'conv_window': (5,), 'batch_size': 32, 'activation': 'relu'}
0.972568 (0.000964) with: {'pooling_window': (2,), 'optimizer': ('adam',

In [18]:
cnn = network(activation='relu', conv_window=(3,), dense_activation='sigmoid', dropout_rate=[0.15344147, 0.10309187, 0.05869609, 0.61434385, 0.66912969], filter_number=20, input_window_size=60, loss='categorical_crossentropy', optimizer=['adam'], pooling_window=(2,))

In [ ]:
cnn.fit(x_train, y_train, epochs=30, batch_size=32)